In [1]:
from dart_fss import get_corp_list
from dotenv import load_dotenv
import dart_fss as dart
import pandas as pd
import requests
import json
import os
import time

load_dotenv()

API_KEY = os.getenv("API_KEY")

dart.set_api_key(api_key=API_KEY)

# 모든 상장된 기업 리스트 불러오기
corp_list = get_corp_list()


# 삼성전자를 이름으로 찾기 ( 리스트 반환 )
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]
samsung

Error occurred during getting browser(s): random, but was suppressed with fallback.


Label,Data
corp_code,00126380
corp_name,삼성전자
corp_eng_name,"SAMSUNG ELECTRONICS CO,.LTD"
stock_code,005930
modify_date,20250326
sector,통신 및 방송 장비 제조업
product,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조(디스플레이) 제품, 영상 및 음향기기 제조(영상기기) 제품 등"
corp_cls,Y


In [28]:
industry = "정보통신업" 

df_kospi = pd.read_excel(f"업종별_기업리스트/{industry}_KOSPI.xlsx")
df_kosdaq = pd.read_excel(f"업종별_기업리스트/{industry}_KOSDAQ.xlsx")

df_total = pd.concat([df_kospi, df_kosdaq], ignore_index=True)
df_total = df_total.drop_duplicates()
df_total["업종구분"] = industry

corp_name_list = list(df_total["회사명"])
print(f"{industry} 기업 목록을 리스트로 저장했습니다.")
# corp_name_list

정보통신업 기업 목록을 리스트로 저장했습니다.


c:\Users\82107\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\82107\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [30]:
target_ratio = [
    "자기자본비율", "부채비율", "유동비율", "유동부채비율", # 안정성지표
    "매출액영업이익률", "총자산순이익률", # 수익성지표
    "매출액증가율", # 성장성지표
    "총자산회전율" # 활동성지표
]

idx_cl_code_dict = {
    # "M210000": "수익성지표",
    "M220000": "안정성지표",
    # "M230000": "성장성지표",
    # "M240000": "활동성지표"
}

corp_name_dict = {}

for corp in corp_name_list:
    name = corp_list.find_by_corp_name(corp, exactly=True)
    if name:
        corp_code = str(name[0])[1:9]
    else:
        print(f"{corp}(은)는 다중회사가 아니므로 건너뜁니다.")
        continue
    corp_name_dict[f"{corp}"] = corp_code
    # print(f"{corp}의 회사 이름과 고유번호를 딕셔너리에 추가했습니다.")

print(f"회사는 총 {len(corp_name_dict)}개입니다.")
# print(f"회사명&고유번호 딕셔너리:\n{corp_name_dict}")

# 기업별 종합 row
corp_data_dict = {}

회사는 총 322개입니다.


In [35]:
# --- API 요청 URL 및 파라미터 ---
for idx_cl_code, 지표분류 in idx_cl_code_dict.items():

    print(f"=== {지표분류} 요청 중 ===")
    
    n = 1

    for corp_name, corp_code in corp_name_dict.items():
        base_url = "https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json"
        params = {
            'crtfc_key': API_KEY,
            'corp_code': corp_code, # 대상 기업 코드
            'bsns_year': "2024", # 회계연도
            'reprt_code': "11011", # 사업보고서
            'idx_cl_code': idx_cl_code,
        }

        # --- GET 요청 보내기 ---
        try:
            response = requests.get(base_url, params=params)
            
            # --- 응답 확인 ---
            # HTTP 상태 코드가 200 (OK)인지 확인
            if response.status_code == 200:
                # print(response.text) 분석&코드 작성 시 사용
                
                data = json.loads(response.text)

                # DART API 응답 자체의 상태 확인 (status가 "000"이 아니면 정상)
                if data.get("status") != "000":
                    print(f"{corp_name} 데이터 조회 실패: {data.get('message')} {n}/{len(corp_name_dict)}")
                    time.sleep(0.3) # 오류 발생 시에도 잠시 대기
                    n += 1
                    continue # 다음 기업으로 넘어감

                # 한 기업 당 하나의 행 생성
                if corp_name not in corp_data_dict:
                    corp_data_dict[corp_name] = {
                        "기업명": corp_name,
                    }

                for item in data.get("list", []):
                    idx_nm = item.get("idx_nm")
                    if idx_nm in target_ratio:
                        corp_data_dict[corp_name][idx_nm] = item.get("idx_val")
                
                print(f"{corp_name}의 주요지표를 DataFrame에 추가했습니다. {n}/{len(corp_name_dict)}")
                n += 1

            else:
                print(f"에러 발생: HTTP 상태 코드 {response.status_code}")

            time.sleep(0.3)

        except requests.exceptions.RequestException as e:
            print(f"요청 중 에러 발생: {e}")

=== 안정성지표 요청 중 ===
CJ CGV의 주요지표를 DataFrame에 추가했습니다. 1/322
DB의 주요지표를 DataFrame에 추가했습니다. 2/322
KTcs의 주요지표를 DataFrame에 추가했습니다. 3/322
KTis의 주요지표를 DataFrame에 추가했습니다. 4/322
LG씨엔에스의 주요지표를 DataFrame에 추가했습니다. 5/322
LG유플러스의 주요지표를 DataFrame에 추가했습니다. 6/322
LG헬로비전의 주요지표를 DataFrame에 추가했습니다. 7/322
NAVER의 주요지표를 DataFrame에 추가했습니다. 8/322
NICE평가정보의 주요지표를 DataFrame에 추가했습니다. 9/322
SBS의 주요지표를 DataFrame에 추가했습니다. 10/322
SK텔레콤의 주요지표를 DataFrame에 추가했습니다. 11/322
YG PLUS의 주요지표를 DataFrame에 추가했습니다. 12/322
넷마블 데이터 조회 실패: 조회된 데이타가 없습니다. 13/322
다우기술의 주요지표를 DataFrame에 추가했습니다. 14/322
더블유게임즈의 주요지표를 DataFrame에 추가했습니다. 15/322
더존비즈온의 주요지표를 DataFrame에 추가했습니다. 16/322
롯데이노베이트의 주요지표를 DataFrame에 추가했습니다. 17/322
미래아이앤지의 주요지표를 DataFrame에 추가했습니다. 18/322
비상교육의 주요지표를 DataFrame에 추가했습니다. 19/322
삼성에스디에스의 주요지표를 DataFrame에 추가했습니다. 20/322
삼성출판사의 주요지표를 DataFrame에 추가했습니다. 21/322
시프트업의 주요지표를 DataFrame에 추가했습니다. 22/322
신세계I&C의 주요지표를 DataFrame에 추가했습니다. 23/322
아센디오의 주요지표를 DataFrame에 추가했습니다. 24/322
아시아나IDT의 주요지표를 DataFrame에 추가했습니다. 25/322
아이에이치큐의 주요

KeyboardInterrupt: 

In [34]:
# DataFrame 생성
df = pd.DataFrame(list(corp_data_dict.values()))
df

# CSV 저장
# df.to_csv("주요지표.csv", index=False, encoding='utf-8-sig')

,기업명,자기자본비율,부채비율,유동비율,유동부채비율
0,CJ CGV,14.429,593.03,45.731,295.31
1,DB,45.316,120.672,87.947,82.637
2,KTcs,46.645,114.384,155.935,93.575
3,KTis,44.284,125.814,104.993,59.047
4,LG씨엔에스,47.126,112.195,185.46,87.258
...,...,...,...,...,...
278,헥토이노베이션,47.637,109.922,157.214,103.9
279,헥토파이낸셜,33.519,198.339,125.782,195.203
280,현대이지웰,43.176,131.611,149.388,117.266
281,휴네시온,86.034,16.233,278.805,15.517
